In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb

In [2]:
train = pd.read_csv('processed_data/train_clean.csv')
#test = pd.read_csv('processed_data/test_clean.csv')

In [3]:
train.head().T

,0,1,2,3,4
srch_id,1,1,1,1,1
date_time,2013-04-04 08:32:15,2013-04-04 08:32:15,2013-04-04 08:32:15,2013-04-04 08:32:15,2013-04-04 08:32:15
site_id,12,12,12,12,12
visitor_location_country_id,187,187,187,187,187
visitor_hist_starrating,NaN,NaN,NaN,NaN,NaN
visitor_hist_adr_usd,NaN,NaN,NaN,NaN,NaN
prop_country_id,219,219,219,219,219
prop_id,893,10404,21315,27348,29604
prop_starrating,3,4,3,2,4
prop_review_score,3.5,4,4.5,4,3.5


In [8]:
train.booking_bool.value_counts()/len(train)

0    0.972089
1    0.027911
Name: booking_bool, dtype: float64

In [4]:
#booked = train.booking_bool
#clicked = train.click_bool
#gross_usd = train.gross_bookings_usd
#year = pd.DatetimeIndex(train['date_time']).year
#month = pd.DatetimeIndex(train['date_time']).month
# drop some columns
train_new = train.drop(['booking_bool','click_bool','gross_bookings_usd','date_time','position'], axis=1)

# set label column, where 1 - clicked, 2 - booked
#train_new.insert(0, 'label',(clicked+booked))

In [5]:
unique_srch_ids = train_new.srch_id.unique()
rand_ind = np.random.randint(0,len(unique_srch_ids), len(unique_srch_ids)//2)
train_ind = unique_srch_ids[rand_ind]

# split into train and test 
train_set = train_new[train_new.srch_id.isin(train_ind)]
test_set = train_new[~train_new.srch_id.isin(train_ind)]

train_set = train_set.sort_values(by=['srch_id'])
test_set = test_set.sort_values(by=['srch_id'])

train_label = train_set.booked_clicked_combined.values
test_label = test_set.booked_clicked_combined.values
train_set_nolabel = train_set.drop(['booked_clicked_combined'], axis=1)
test_set_nolabel = test_set.drop(['booked_clicked_combined'], axis=1)

In [6]:
train_set_nolabel.fillna(-999, inplace=True)
test_set_nolabel.fillna(-999, inplace=True)

In [7]:
# get the frequencies for srch_id to set 'group' parameter in lightGBM Dataset
train_group = train_set_nolabel.groupby(['srch_id']).size().values
test_group = test_set_nolabel.groupby(['srch_id']).size().values

# make datasets for lightGBM
lgb_train = lgb.Dataset(train_set_nolabel.as_matrix(),label=train_label, group=train_group)
lgb_val = lgb.Dataset(test_set_nolabel.as_matrix(), label=test_label, group=test_group)

In [12]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'lambdarank',
    'metric': 'ndcg',
    'label_gain' : {0,1,5},
    'ndcg_at': 38,
    'num_leaves': 10,
    'min_data_in_leaf':70,
    'num_iterations':500,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'verbose': 0
}
lgbm_model = lgb.train(params, train_set = lgb_train, valid_sets = lgb_val, verbose_eval=50)

/home/sesegma/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[50]	valid_0's ndcg@38: 0.502344
[100]	valid_0's ndcg@38: 0.509027
[150]	valid_0's ndcg@38: 0.51453
[200]	valid_0's ndcg@38: 0.517963
[250]	valid_0's ndcg@38: 0.520801
[300]	valid_0's ndcg@38: 0.522758
[350]	valid_0's ndcg@38: 0.524425
[400]	valid_0's ndcg@38: 0.525767
[450]	valid_0's ndcg@38: 0.526806
[500]	valid_0's ndcg@38: 0.527658
